# Are there factors that can affect one's eating habits?

In [719]:
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
from sklearn import linear_model
import statsmodels.formula.api as smf
from sqlalchemy import create_engine
import seaborn as sns
from sklearn.linear_model import LinearRegression
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_absolute_error
import statsmodels.api as sm
from statsmodels.tools.eval_measures import mse, rmse

from sqlalchemy import create_engine

In [720]:
pd.set_option('display.max_rows', 3000)

In [721]:
# Importing the data into a dataframe

df = pd.read_csv('food_coded.csv')


In [722]:
weight_df =  df[['eating_changes_coded1','GPA','exercise','cook','eating_out','employment','income','life_rewarding','sports','weight']]

weight_df


,eating_changes_coded1,GPA,exercise,cook,eating_out,employment,income,life_rewarding,sports,weight
0,1,2.4,1.0,2.0,3,3.0,5.0,1.0,1.0,187
1,2,3.654,1.0,3.0,2,2.0,4.0,1.0,1.0,155
2,3,3.3,2.0,1.0,2,3.0,6.0,7.0,2.0,I'm not answering this.
3,3,3.2,3.0,2.0,2,3.0,6.0,2.0,2.0,"Not sure, 240"
4,4,3.5,1.0,1.0,2,2.0,6.0,1.0,1.0,190
5,3,2.25,2.0,3.0,1,3.0,1.0,4.0,2.0,190
6,5,3.8,1.0,2.0,2,3.0,4.0,8.0,1.0,180
7,5,3.3,2.0,3.0,2,2.0,5.0,3.0,2.0,137
8,8,3.3,NaN,3.0,5,2.0,5.0,8.0,2.0,180
9,3,3.3,1.0,3.0,3,3.0,4.0,3.0,1.0,125


In [723]:
weight_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 125 entries, 0 to 124
Data columns (total 10 columns):
eating_changes_coded1    125 non-null int64
GPA                      123 non-null object
exercise                 112 non-null float64
cook                     122 non-null float64
eating_out               125 non-null int64
employment               116 non-null float64
income                   124 non-null float64
life_rewarding           124 non-null float64
sports                   123 non-null float64
weight                   123 non-null object
dtypes: float64(6), int64(2), object(2)
memory usage: 9.8+ KB


In [724]:
# checking to see which columns do not have numerical values

weight_df_null = weight_df.isnull().sum()*100/weight_df.isnull().count()
weight_df_null.sort_values()


eating_changes_coded1     0.0
eating_out                0.0
income                    0.8
life_rewarding            0.8
GPA                       1.6
sports                    1.6
weight                    1.6
cook                      2.4
employment                7.2
exercise                 10.4
dtype: float64

In [725]:
new_exercise_column = pd.DataFrame(weight_df['exercise'].fillna(weight_df['exercise'].mean(),inplace=False))
new_exercise_column = np.round(new_exercise_column, decimals=2)


weight_df = weight_df.drop('exercise', axis=1)
weight_df.insert(2,'exercise',new_exercise_column, True)


weight_df.update(weight_df.replace('3.79 bitch',3.79, inplace=False))
weight_df['GPA'] = weight_df['GPA'].drop([61, 104], inplace=False)
weight_df['GPA'] = weight_df['GPA'].astype(float)

new_GPA_column = pd.DataFrame(weight_df['GPA'].fillna(weight_df['GPA'].mean(),inplace=False))
new_GPA_column = np.round(new_GPA_column, decimals=2)

weight_df = weight_df.drop('GPA', axis=1)
weight_df.insert(1,'GPA',new_GPA_column, True)


In [726]:
new_employment_column    = pd.DataFrame(weight_df['employment'].fillna(weight_df['employment'].mean(),inplace=False))
new_income_column        = pd.DataFrame(weight_df['income'].fillna(weight_df['income'].mean(),inplace=False))
new_lifeRewarding_column = pd.DataFrame(weight_df['life_rewarding'].fillna(weight_df['life_rewarding'].mean(),inplace=False))
new_sports_column        = pd.DataFrame(weight_df['sports'].fillna(weight_df['sports'].mean(),inplace=False))
new_cook_column          = pd.DataFrame(weight_df['cook'].fillna(weight_df['cook'].mean(),inplace=False))

new_employment_column = np.round(new_employment_column, decimals=1)
new_income_column = np.round(new_income_column, decimals=1)
new_lifeRewarding_column = np.round(new_lifeRewarding_column, decimals=1)
new_sports_column = np.round(new_sports_column, decimals=1)
new_cook_column  = np.round(new_cook_column, decimals=1)

weight_df = weight_df.drop('employment', axis=1)
weight_df = weight_df.drop('income', axis=1)
weight_df = weight_df.drop('life_rewarding', axis=1)
weight_df = weight_df.drop('sports', axis=1)
weight_df = weight_df.drop('cook', axis=1)

weight_df.insert(5,'employment',new_employment_column, True)
weight_df.insert(6,'income',new_income_column, True)
weight_df.insert(7,'life_rewarding',new_lifeRewarding_column, True)
weight_df.insert(8,'sports',new_sports_column, True)
weight_df.insert(3,'cook',new_cook_column,True )


weight_df.update(weight_df.replace('Not sure, 240',240, inplace=False))
weight_df.update(weight_df.replace('144 lbs',144, inplace=False))

weight_df['weight'] = weight_df['weight'].drop([2], inplace=False)





In [727]:
weight_df

,eating_changes_coded1,GPA,exercise,cook,eating_out,weight,employment,income,life_rewarding,sports
0,1,2.40,1.00,2.0,3,187,3.0,5.0,1.0,1.0
1,2,3.65,1.00,3.0,2,155,2.0,4.0,1.0,1.0
2,3,3.30,2.00,1.0,2,NaN,3.0,6.0,7.0,2.0
3,3,3.20,3.00,2.0,2,240,3.0,6.0,2.0,2.0
4,4,3.50,1.00,1.0,2,190,2.0,6.0,1.0,1.0
5,3,2.25,2.00,3.0,1,190,3.0,1.0,4.0,2.0
6,5,3.80,1.00,2.0,2,180,3.0,4.0,8.0,1.0
7,5,3.30,2.00,3.0,2,137,2.0,5.0,3.0,2.0
8,8,3.30,1.59,3.0,5,180,2.0,5.0,8.0,2.0
9,3,3.30,1.00,3.0,3,125,3.0,4.0,3.0,1.0


In [728]:
new_weight_column = weight_df['weight'].fillna(0)

In [730]:
new_weight_column

0      187
1      155
2        0
3      240
4      190
5      190
6      180
7      137
8      180
9      125
10     116
11     110
12     264
13     123
14     185
15     180
16     145
17     170
18     135
19     165
20     175
21     195
22     185
23     185
24     105
25     125
26     160
27     175
28     180
29     167
30     115
31     205
32       0
33     128
34     150
35     150
36     150
37     170
38     150
39     175
40     140
41     120
42     135
43     100
44     170
45     113
46     168
47     145
48     155
49     150
50     169
51     185
52     200
53     265
54     165
55     192
56     175
57     140
58     155
59     155
60     135
61     118
62     210
63     180
64     140
65     112
66     125
67     144
68     145
69     130
70     140
71     140
72     140
73     200
74       0
75     120
76     150
77     200
78     135
79     145
80     130
81     190
82     170
83     127
84     167
85     140
86     190
87     155
88     175
89     129
90     260

In [733]:
weight_df['weight'].update(new_weight_column)


In [734]:
weight_df["weight"] = weight_df["weight"].astype(int)



In [741]:
last_updated_weight = weight_df['weight'].replace(0, weight_df['weight'].mean(), inplace=False )


In [742]:
weight_df['weight'].update(last_updated_weight)


In [743]:
weight_df

,eating_changes_coded1,GPA,exercise,cook,eating_out,weight,employment,income,life_rewarding,sports
0,1,2.40,1.00,2.0,3,187.000,3.0,5.0,1.0,1.0
1,2,3.65,1.00,3.0,2,155.000,2.0,4.0,1.0,1.0
2,3,3.30,2.00,1.0,2,155.232,3.0,6.0,7.0,2.0
3,3,3.20,3.00,2.0,2,240.000,3.0,6.0,2.0,2.0
4,4,3.50,1.00,1.0,2,190.000,2.0,6.0,1.0,1.0
5,3,2.25,2.00,3.0,1,190.000,3.0,1.0,4.0,2.0
6,5,3.80,1.00,2.0,2,180.000,3.0,4.0,8.0,1.0
7,5,3.30,2.00,3.0,2,137.000,2.0,5.0,3.0,2.0
8,8,3.30,1.59,3.0,5,180.000,2.0,5.0,8.0,2.0
9,3,3.30,1.00,3.0,3,125.000,3.0,4.0,3.0,1.0


In [688]:
for column_name in weight_df['weight']:
    print("These are the problematic values for the variable: {}".format(column_name))
    for value in weight_df[column_name]:
        try:
            float(value)
        except:
            print(value)


# look into cleaning type_sports so you can make dummy variables or simply clean up till you can use as a feature 

In [696]:
for column_name in ['eating_changes_coded1','exercise','cook','grade_level','income','sports','GPA','Gender']:
    print("These are the problematic values for the variable: {}".format(column_name))
    for value in df[column_name]:
        try:
            float(value)
        except:
            print(value)


In [697]:
for column_name in ['exercise','eating_changes','cook','grade_level','income','sports','GPA','Gender']:
    print("These are the problematic values for the variable: {}".format(column_name))
    for value in df[column_name]:
        try:
            float(value)
        except:
            print(value)
